In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import akshare as ak
import json
import os
import time

from datetime import datetime, timedelta
from tqdm import tqdm

pd.options.display.max_rows=None
pd.options.display.max_columns=None

In [139]:
# 1. 获取A股所有股票列表
# stock_code_list = ak.stock_info_a_code_name()['code']
stock_code_list = ak.index_stock_cons("000016")['品种代码'].tolist() # 获取中证50的股票代码列表

## 1. 获取基础原始数据

In [141]:
def get_stock_history_info(stock_code):
    result = ak.stock_zh_a_hist(symbol=stock_code, adjust='hfq').rename(
            columns={
                "日期": "datetime",
                "开盘": "open",
                "最高": "high",
                "最低": "low",
                "收盘": "close",
                "成交量": "volume",
                "成交额": "turnover",
                "振幅": "amplitude",
                "涨跌幅": "change_pct",
                "涨跌额": "change_amount",
                "换手率": "turnover_rate",
            }
        )
    result = result[['datetime', 'open', 'high', 'low', 'close', 'volume']]
    result.insert(0, 'stock_code', stock_code)
    return result

def get_stock_label(dataframe, expression_excutor):
    # 计算收益：5日收盘价(作为卖出价格)除以明日开盘价(作为买入价格)
    expression_1 = "shift(close,-5)/shift(open,-1)-1"
    # 极值处理：用1%和99%分位的值做clip
    expression_2 = "clip(label,all_quantile(label, 0.01),all_quantile(label,0.99))"
    # 过滤掉一字涨停的情况 (设置label为NaN，在后续处理和训练中会忽略NaN的label)
    expression_3 = "where(shift(high,-1)=shift(low,-1), nan, label)"

    dataframe['label'] = expression_excutor.excute(dataframe, expression_1)
    dataframe['label'] = expression_excutor.excute(dataframe, expression_2)
    dataframe['label'] = expression_excutor.excute(dataframe, expression_3)
    return dataframe

def get_basic_factor(dataframe, expression_excutor):
    alpha_dict = json.loads(open('./alpha_184.json', "r").read())
    for alpha_name, alpha_expression in alpha_dict.items():
        dataframe[alpha_name] = expression_excutor.excute(dataframe, alpha_expression)
    return dataframe

In [143]:
from expression_excutor import AlphaExpressionExcutor
expression_excutor = AlphaExpressionExcutor()

stock_data_list = []
for stock_code in tqdm(stock_code_list):
    stock_data = get_stock_history_info(stock_code)
    stock_data = get_stock_label(stock_data, expression_excutor)
    stock_data = get_basic_factor(stock_data, expression_excutor)
    stock_data = stock_data.replace([np.inf, -np.inf], np.nan).dropna()
    stock_data_list.append(stock_data)

100%|██████████| 50/50 [01:34<00:00,  1.88s/it]


In [144]:
df = pd.concat(stock_data_list)
print([i for i in df.columns])

['stock_code', 'datetime', 'open', 'high', 'low', 'close', 'volume', 'label', 'KMID', 'KLEN', 'KMID2', 'KUP', 'KUP2', 'KLOW', 'KLOW2', 'KSFT', 'KSFT2', 'OPEN0', 'OPEN1', 'OPEN2', 'OPEN3', 'OPEN4', 'HIGH0', 'HIGH1', 'HIGH2', 'HIGH3', 'HIGH4', 'LOW0', 'LOW1', 'LOW2', 'LOW3', 'LOW4', 'CLOSE0', 'CLOSE1', 'CLOSE2', 'CLOSE3', 'CLOSE4', 'VOLUME0', 'VOLUME1', 'VOLUME2', 'VOLUME3', 'VOLUME4', 'ROC5', 'ROC10', 'ROC20', 'ROC30', 'ROC60', 'MAX5', 'MAX10', 'MAX20', 'MAX30', 'MAX60', 'MIN5', 'MIN10', 'MIN20', 'MIN30', 'MIN60', 'MA5', 'MA10', 'MA20', 'MA30', 'MA60', 'STD5', 'STD10', 'STD20', 'STD30', 'STD60', 'BETA5', 'BETA10', 'BETA20', 'BETA30', 'BETA60', 'RSQR5', 'RSQR10', 'RSQR20', 'RSQR30', 'RSQR60', 'RESI5', 'RESI10', 'RESI20', 'RESI30', 'RESI60', 'QTLU5', 'QTLU10', 'QTLU20', 'QTLU30', 'QTLU60', 'QTLD5', 'QTLD10', 'QTLD20', 'QTLD30', 'QTLD60', 'TSRANK5', 'TSRANK10', 'TSRANK20', 'TSRANK30', 'TSRANK60', 'RSV5', 'RSV10', 'RSV20', 'RSV30', 'RSV60', 'IMAX5', 'IMAX10', 'IMAX20', 'IMAX30', 'IMAX60', '

## 4. 模型训练

In [159]:
# 6. 选择固定时间区间的数据
train_start_date = pd.to_datetime('2010-01-01')
train_end_date = pd.to_datetime('2019-12-31')
val_start_date = pd.to_datetime('2020-01-01')
val_end_date = pd.to_datetime('2020-12-31')

train_data = df[(pd.to_datetime(df['datetime']) >= train_start_date) & (pd.to_datetime(df['datetime']) <= train_end_date)]
validation_data = df[(pd.to_datetime(df['datetime']) >= val_start_date) & (pd.to_datetime(df['datetime']) <= val_end_date)]

print(f"train_data_size: {train_data.shape}")
print(f"validation_data_size: {validation_data.shape}")

train_data_size: (93655, 182)
validation_data_size: (11027, 182)


## 使用Tensorflow

In [160]:
# 使用tensorflow处理原始数据
import numpy as np
import pandas as pd
import tensorflow as tf
print(tf.__version__)

2.15.0


In [174]:
def get_numeric_boundaries(series, num_bins=20):
    if series.nunique() < num_bins:
        boundaries = sorted(series.unique())
    else:
        boundaries = pd.qcut(series, num_bins, retbins=True, duplicates='drop')[1].tolist()
    return boundaries

In [175]:
TARGET_FEATURE_NAME = ["label"]

NUMERIC_FEATURES = ['KMID', 'KLEN', 'KMID2', 'KUP', 'KUP2', 'KLOW', 'KLOW2', 'KSFT', 'KSFT2', 'OPEN0', 'OPEN1', 'OPEN2', 'OPEN3', 'OPEN4', 'HIGH0', 'HIGH1', 'HIGH2', 'HIGH3', 'HIGH4', 'LOW0', 'LOW1', 'LOW2', 'LOW3', 'LOW4', 'CLOSE0', 'CLOSE1', 'CLOSE2', 'CLOSE3', 'CLOSE4', 'VOLUME0', 'VOLUME1', 'VOLUME2', 'VOLUME3', 'VOLUME4', 'ROC5', 'ROC10', 'ROC20', 'ROC30', 'ROC60', 'MAX5', 'MAX10', 'MAX20', 'MAX30', 'MAX60', 'MIN5', 'MIN10', 'MIN20', 'MIN30', 'MIN60', 'MA5', 'MA10', 'MA20', 'MA30', 'MA60', 'STD5', 'STD10', 'STD20', 'STD30', 'STD60', 'BETA5', 'BETA10', 'BETA20', 'BETA30', 'BETA60', 'RSQR5', 'RSQR10', 'RSQR20', 'RSQR30', 'RSQR60', 'RESI5', 'RESI10', 'RESI20', 'RESI30', 'RESI60', 'QTLU5', 'QTLU10', 'QTLU20', 'QTLU30', 'QTLU60', 'QTLD5', 'QTLD10', 'QTLD20', 'QTLD30', 'QTLD60', 'TSRANK5', 'TSRANK10', 'TSRANK20', 'TSRANK30', 'TSRANK60', 'RSV5', 'RSV10', 'RSV20', 'RSV30', 'RSV60', 'IMAX5', 'IMAX10', 'IMAX20', 'IMAX30', 'IMAX60', 'IMIN5', 'IMIN10', 'IMIN20', 'IMIN30', 'IMIN60', 'IMXD5', 'IMXD10', 'IMXD20', 'IMXD30', 'IMXD60', 'CORD5', 'CORD10', 'CORD20', 'CORD30', 'CORD60', 'CNTP5', 'CNTP10', 'CNTP20', 'CNTP30', 'CNTP60', 'CNTN5', 'CNTN10', 'CNTN20', 'CNTN30', 'CNTN60', 'CNTD5', 'CNTD10', 'CNTD20', 'CNTD30', 'CNTD60', 'SUMP5', 'SUMP10', 'SUMP20', 'SUMP30', 'SUMP60', 'SUMN5', 'SUMN10', 'SUMN20', 'SUMN30', 'SUMN60', 'SUMD5', 'SUMD10', 'SUMD20', 'SUMD30', 'SUMD60', 'VMA5', 'VMA10', 'VMA20', 'VMA30', 'VMA60', 'VSTD5', 'VSTD10', 'VSTD20', 'VSTD30', 'VSTD60', 'WVMA5', 'WVMA10', 'WVMA20', 'WVMA30', 'WVMA60', 'VSUMP5', 'VSUMP10', 'VSUMP20', 'VSUMP30', 'VSUMP60', 'VSUMN5', 'VSUMN10', 'VSUMN20', 'VSUMN30', 'VSUMN60', 'VSUMD5', 'VSUMD10', 'VSUMD20', 'VSUMD30', 'VSUMD60']
NUMERIC_FEATURES_WITH_BOUNDARIES = {k: get_numeric_boundaries(train_data[k])  for k in NUMERIC_FEATURES}

INTEGER_CATEGORICAL_FEATURES = []
INTEGER_CATEGORICAL_FEATURES_WITH_VOCAB = {}

STRING_CATEGORICAL_FEATURES = []
STRING_CATEGORICAL_FEATURES_WITH_VOCAB = {}

FEATURE_NAMES = NUMERIC_FEATURES + INTEGER_CATEGORICAL_FEATURES + STRING_CATEGORICAL_FEATURES

In [176]:
class Senet(tf.keras.layers.Layer):
    def __init__(self, reduction_ratio=3, seed=1024, **kwargs):
        super(Senet, self).__init__(**kwargs)
        self.reduction_ratio = reduction_ratio
        self.seed = seed  

    def build(self, input_shape):
        self.field_size = len(input_shape)
        self.reduction_size = max(1, self.field_size // self.reduction_ratio)
        self.scale_layer = tf.keras.layers.Dense(units=self.reduction_size, activation='relu')
        self.expand_layer = tf.keras.layers.Dense(units=self.field_size, activation='relu')
        super(Senet, self).build(input_shape)

    def call(self, inputs, training=None):
        inputs = [tf.expand_dims(i, axis=1) for i in inputs]
        inputs = tf.concat(inputs, axis=1) # [B, N, dim]
        Z = tf.reduce_mean(inputs, axis=-1) # [B, N]
        A_1 = self.scale_layer(Z) # [B, X]
        A_2 = self.expand_layer(A_1) # [B, N]
        scale_inputs = tf.multiply(inputs, tf.expand_dims(A_2, axis=-1))
        output = scale_inputs + inputs # skip-connection
        return output # [B, N, dim]


class Dnn(tf.keras.layers.Layer):
    def __init__(self, hidden_units, activation="relu", dropout_rate=0.2, use_bn=False, seed=1024, **kwargs):
        super(Dnn, self).__init__(**kwargs)
        self.hidden_units = hidden_units
        self.activation = activation
        self.dropout_rate = dropout_rate
        self.use_bn = use_bn
        self.seed = seed
        self.dense_layers = []
        self.dropout_layers = []
        self.bn_layers = []
        
    def build(self, input_shape):
        for units in self.hidden_units:
            self.dense_layers.append(tf.keras.layers.Dense(units=units, activation=self.activation))
            self.dropout_layers.append(tf.keras.layers.Dropout(rate=self.dropout_rate, seed=self.seed))
            if self.use_bn:
                self.bn_layers.append(tf.keras.layers.BatchNormalization())
        super(Dnn, self).build(input_shape)  # Be sure to call this at the end
    
    def call(self, inputs, training=None):
        x = inputs
        for i in range(len(self.hidden_units)):
            x = self.dense_layers[i](x)
            if self.use_bn:
                x = self.bn_layers[i](x, training=training)
            x = self.dropout_layers[i](x, training=training)
        return x

In [177]:
class QuantModel(tf.keras.Model):
	def __init__(self, config, **kwargs):
		super(QuantModel, self).__init__(**kwargs)
		self.config = config

		# 添加属性来存储预定义的层
		self.lookup_layers = {}
		self.embedding_layers = {}

        # 创建连续特征的离散化层和嵌入层
		for feature_name, boundaries in self.config.get("numeric_features_with_boundaries").items():
			self.lookup_layers[feature_name] = tf.keras.layers.Discretization(bin_boundaries=boundaries, output_mode='int', name=f'{feature_name}_lookup')
			self.embedding_layers[feature_name] = tf.keras.layers.Embedding(input_dim=len(boundaries) + 1, output_dim=self.config.get("feature_embedding_dims", 6), name=f'{feature_name}_embedding')
        # 创建整数特征的查找层和嵌入层
		for feature_name, vocab in self.config.get("integer_categorical_features_with_vocab").items():
			self.lookup_layers[feature_name] = tf.keras.layers.IntegerLookup(vocabulary=vocab, name=f'{feature_name}_lookup')
			self.embedding_layers[feature_name] = tf.keras.layers.Embedding(input_dim=len(vocab) + 1, output_dim=self.config.get("feature_embedding_dims", 6), name=f'{feature_name}_embedding')
		# 创建字符串特征的查找层和嵌入层
		for feature_name, vocab in self.config.get("string_categorical_features_with_vocab").items():
			self.lookup_layers[feature_name] = tf.keras.layers.StringLookup(vocabulary=vocab, name=f'{feature_name}_lookup')
			self.embedding_layers[feature_name] = tf.keras.layers.Embedding(input_dim=len(vocab) + 1, output_dim=self.config.get("feature_embedding_dims", 6), name=f'{feature_name}_embedding')

		self.senet_layer = Senet(
			reduction_ratio=self.config.get('reduction_ratio', 3), 
			seed=self.config.get('seed', 1024),
		)
		self.dnn_layer = Dnn(
			hidden_units=self.config.get('dnn_hidden_units', [64,32]),
			activation=self.config.get('dnn_activation', 'relu'),
			dropout_rate=self.config.get('dnn_dropout', 0.2),
			use_bn=self.config.get('dnn_use_bn', True)
		)
		self.output_layer = tf.keras.layers.Dense(1, activation=None)
		

	def call(self, inputs, training=None):
		# 确保inputs是一个字典类型，每个键值对应一个特征输入
		if not isinstance(inputs, dict): 
			raise ValueError('The inputs to the model should be a dictionary where keys are feature names.')
		encoded_features = []
    	# 现在使用已经实例化的层来编码输入
		for feature_name in inputs:
        	# 使用预定义的查找层和嵌入层
			lookup_layer = self.lookup_layers[feature_name]
			embedding_layer = self.embedding_layers[feature_name]
			encoded_feature = embedding_layer(lookup_layer(inputs[feature_name]))
			encoded_features.append(encoded_feature)

		senet_output = self.senet_layer(encoded_features, training=training)
		senet_output = tf.keras.layers.Flatten()(senet_output) # [B, N * dim]
		dnn_output = self.dnn_layer(senet_output, training=training)
		output = self.output_layer(dnn_output, training=training)
		return output

In [178]:
def df_to_dataset(dataframe, feature_cols, label_cols, shuffle=True, batch_size=32):
  features = dataframe[feature_cols]
  labels = dataframe[label_cols]
  ds = tf.data.Dataset.from_tensor_slices((dict(features), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(features))
  ds = ds.batch(batch_size)
  ds = ds.prefetch(batch_size)
  return ds

train_ds = df_to_dataset(train_data, FEATURE_NAMES, TARGET_FEATURE_NAME, shuffle=True)
val_ds = df_to_dataset(validation_data, FEATURE_NAMES, TARGET_FEATURE_NAME, shuffle=False)

In [179]:
model_config = {
    "seed": 1024,
    "reduction_ratio": 3,
    "dnn_hidden_units": [64,32],
    "dnn_activation": 'relu',
    "dnn_dropout": 0.2,
    "dnn_use_bn": True,
    "numeric_features_with_boundaries": NUMERIC_FEATURES_WITH_BOUNDARIES,
    "integer_categorical_features_with_vocab": INTEGER_CATEGORICAL_FEATURES_WITH_VOCAB,
    "string_categorical_features_with_vocab": STRING_CATEGORICAL_FEATURES_WITH_VOCAB,
    "feature_embedding_dims": 6
}

model = QuantModel(model_config)

optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=1e-3)
loss = tf.keras.losses.MeanSquaredError()
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    verbose=1,
    patience=10,
    mode='max',
    restore_best_weights=True,
)

model.compile(optimizer, loss=loss)
model.fit(
        train_ds, 
        validation_data=val_ds, 
        epochs=10,
        verbose=2,
        callbacks=[early_stopping])

Epoch 1/10
2927/2927 - 21s - loss: 0.0867 - val_loss: 0.0029 - 21s/epoch - 7ms/step
Epoch 2/10
2927/2927 - 15s - loss: 0.0023 - val_loss: 0.0032 - 15s/epoch - 5ms/step
Epoch 3/10
2927/2927 - 15s - loss: 0.0023 - val_loss: 0.0030 - 15s/epoch - 5ms/step
Epoch 4/10
2927/2927 - 15s - loss: 0.0023 - val_loss: 0.0030 - 15s/epoch - 5ms/step
Epoch 5/10
2927/2927 - 15s - loss: 0.0023 - val_loss: 0.0030 - 15s/epoch - 5ms/step
Epoch 6/10
2927/2927 - 15s - loss: 0.0022 - val_loss: 0.0030 - 15s/epoch - 5ms/step
Epoch 7/10
2927/2927 - 15s - loss: 0.0022 - val_loss: 0.0030 - 15s/epoch - 5ms/step
Epoch 8/10
2927/2927 - 16s - loss: 0.0021 - val_loss: 0.0031 - 16s/epoch - 5ms/step
Epoch 9/10
2927/2927 - 17s - loss: 0.0021 - val_loss: 0.0031 - 17s/epoch - 6ms/step
Epoch 10/10
2927/2927 - 15s - loss: 0.0020 - val_loss: 0.0031 - 15s/epoch - 5ms/step


In [180]:
# baseline_model.save('./stock_selection_base_model')
# reloaded_model = tf.keras.models.load_model('./stock_selection_base_model')

In [181]:
test_labels = []
for _, labels in val_ds:
    test_labels.extend(labels.numpy().squeeze())

test_predictions = model.predict(val_ds).squeeze().tolist()

345/345 [==============================] - 2s 3ms/step


In [186]:
test_df = pd.DataFrame()
test_df['true_label'] = test_labels
test_df['prediction'] = test_predictions
test_df.sort_values(by=['true_label'], ascending=False).head(20)

,true_label,prediction
5470,0.310501,-0.030083
3016,0.304501,0.007330
5518,0.279538,-0.039140
5468,0.275968,-0.067975
5469,0.260039,-0.022301
2793,0.256914,-0.012903
2795,0.252496,0.014615
3015,0.248619,-0.021992
5204,0.245190,-0.009643
5394,0.245190,-0.002916


In [40]:
# import matplotlib.pyplot as plt

# plt.hist(true_positives_scores, bins=50, alpha=0.5, label='True Positives')
# plt.hist(false_positives_scores, bins=50, alpha=0.5, label='False Positives')
# plt.xlabel('Scores')
# plt.ylabel('Frequency')
# plt.legend()
# plt.show()

In [157]:
# import matplotlib as mpl
# import matplotlib.pyplot as plt
# import numpy as np
# import pandas as pd
# import seaborn as sns
# import sklearn
# from sklearn.metrics import confusion_matrix

# def plot_cm(labels, predictions, p=0.5):
#     cm = confusion_matrix(labels, predictions > p)
#     plt.figure(figsize=(5,5))
#     sns.heatmap(cm, annot=True, fmt="d")
#     plt.title('Confusion matrix @{:.2f}'.format(p))
#     plt.ylabel('Actual label')
#     plt.xlabel('Predicted label')

#     print('Legitimate Transactions Detected (True Negatives): ', cm[0][0])
#     print('Legitimate Transactions Incorrectly Detected (False Positives): ', cm[0][1])
#     print('Fraudulent Transactions Missed (False Negatives): ', cm[1][0])
#     print('Fraudulent Transactions Detected (True Positives): ', cm[1][1])
#     print('Total Fraudulent Transactions: ', np.sum(cm[1]))

# plot_cm(test_labels, test_predictions)
